# 노이즈 복원 모델 학습용을 위한 노이즈 없는 데이터만 추출

In [ ]:
import re
import pandas as pd

# 데이터셋 로드 => 초기 데이터셋
data = pd.read_csv('/data/ephemeral/data/train.csv')

# 필터링된 정상 텍스트를 저장할 리스트
normal_texts = []
id = []

# 통계 정보 => 텍스트 노이즈 분류 기준
total_texts = 0
excluded_due_to_korean_ratio = 0
excluded_due_to_special_ratio = 0
excluded_due_to_abnormal_pattern = 0


for text in data['text']:
    # 텍스트를 문자열로 변환 (NaN 값 처리)
    if not isinstance(text, str):
        continue
    
    total_texts += 1

    # 텍스트에서 공백 제거
    text_no_space = text.replace(' ', '')
    
    # 텍스트 길이 계산 ,공백 및 특수문자 제외
    total_chars = re.findall(r'[A-Za-z0-9가-힣]', text_no_space)
    total_length = len(total_chars)
    if total_length == 0:
        continue
    
    # 한글, 영어, 숫자, 특수 문자 개수 계산
    korean_chars = re.findall(r'[가-힣]', text_no_space)
    special_chars = re.findall(r'[^A-Za-z0-9가-힣\s]', text_no_space)
    
    korean_ratio = len(korean_chars) / total_length
    special_ratio = len(special_chars) / (len(text_no_space))
    
    # 이상 패턴 검사 => 데이터 보고 추가 조건을 정규표현식으로 추가
    abnormal_patterns = [
        r'[A-Za-z]{3,}[0-9]{3,}',       # 영어 3자 이상 + 숫자 3자 이상 (너무 짧은 패턴 제외)
        r'[0-9]{3,}[A-Za-z]{3,}',       # 숫자 3자 이상 + 영어 3자 이상
        r'[A-Za-z]{2,}[^A-Za-z0-9가-힣\s]{2,}',  # 영어 2자 이상 + 특수문자 2자 이상
        r'[^A-Za-z0-9가-힣\s]{2,}[A-Za-z]{2,}',  # 특수문자 2자 이상 + 영어 2자 이상
        r'[A-Za-z]{2,}-[0-9]+-[A-Za-z]{2,}',  # 영어 2자 이상 - 숫자 - 영어 2자 이상
        r'[^A-Za-z0-9가-힣\s]{3,}',    # 연속된 특수 문자 3자 이상
        r'^(?=[^{}\(\*\[\]]*[}\(\*\[][^{}\(\*\[\]]*$)',  # 특정 기호가 하나만 포함된 문자열 필터링
        r'[A-Za-z]{1,2}[^A-Za-z0-9가-힣\s]+[가-힣]',  # 영어 1~2자 + 특수문자 + 한글
        r'[가-힣]+[^A-Za-z0-9가-힣\s]+[A-Za-z]{1,2}',  # 한글 + 특수문자 + 영어 1~2자
        r'[가-힣]+[^A-Za-z0-9가-힣\s]+[가-힣]+[A-Za-z]',  # 한글 + 특수문자 + 한글 + 영어  
        r'[^A-Za-z0-9가-힣\s]+[0-9]+[^A-Za-z0-9가-힣\s]+' # 특수문자 + 숫자 + 특수문자 
        r''
    ]
    
    has_abnormal_pattern = False
    for pattern in abnormal_patterns:
        if re.search(pattern, text_no_space):
            has_abnormal_pattern = True
            break
    
    # 필터링 조건 설정
    if korean_ratio < 0.8:
        excluded_due_to_korean_ratio += 1
        continue
    if special_ratio > 0.1:
        excluded_due_to_special_ratio += 1
        continue
    if has_abnormal_pattern:
        excluded_due_to_abnormal_pattern += 1
        continue

    # 통과한 텍스트 저장
    normal_texts.append(text)
    
    

# 결과를 데이터프레임으로 변환하여 저장
normal_texts_df = pd.DataFrame({'text': normal_texts})
normal_texts_df.to_csv('normal_texts.csv', index=False)

print(f"총 {len(normal_texts)}개의 정상 텍스트를 추출하였습니다.")
print(f"총 텍스트 수: {total_texts}")
print(f"한글 비율로 제외된 텍스트 수: {excluded_due_to_korean_ratio}")
print(f"특수 문자 비율로 제외된 텍스트 수: {excluded_due_to_special_ratio}")
print(f"이상 패턴으로 제외된 텍스트 수: {excluded_due_to_abnormal_pattern}")


총 945개의 정상 텍스트를 추출하였습니다.
총 텍스트 수: 2800
한글 비율로 제외된 텍스트 수: 1609
특수 문자 비율로 제외된 텍스트 수: 184
이상 패턴으로 제외된 텍스트 수: 62


# 라벨링 용 데이터 추출

In [ ]:
import re
import pandas as pd

# 데이터셋 로드 
data = pd.read_csv('/data/ephemeral/data/train.csv')

# 필터링된 정상 텍스트와 ID를 저장할 리스트
normal_texts = []
normal_ids = []

# 노이즈가 있는 텍스트와 ID를 저장할 리스트
noisy_texts = []
noisy_ids = []

# 통계 정보
total_texts = 0
excluded_due_to_korean_ratio = 0
excluded_due_to_special_ratio = 0
excluded_due_to_abnormal_pattern = 0

# 각 텍스트에 대한 분석 및 필터링
for index, row in data.iterrows():
    text = row['text']
    text_id = row['ID']  

    # 텍스트를 문자열로 변환
    if not isinstance(text, str):
        continue
    
    total_texts += 1

    # 텍스트에서 공백 제거
    text_no_space = text.replace(' ', '')
    
    # 텍스트 길이 계산 
    total_chars = re.findall(r'[A-Za-z0-9가-힣]', text_no_space)
    total_length = len(total_chars)
    if total_length == 0:
        continue
    
    # 한글, 영어, 숫자, 특수 문자 개수 계산
    korean_chars = re.findall(r'[가-힣]', text_no_space)
    special_chars = re.findall(r'[^A-Za-z0-9가-힣\s]', text_no_space)
    
    korean_ratio = len(korean_chars) / total_length
    special_ratio = len(special_chars) / len(text_no_space)
    
    # 이상 패턴 검사 
    # 이때 조건을 좀더 원활하게 하여 노이즈 데이터에 노이즈 없는 데이터가 들어가지 않도록 수정 => 학습 시 좀더 원활하게 하기 위해
    abnormal_patterns = [
        r'[A-Za-z]{3,}[0-9]{3,}',       # 영어 3자 이상 + 숫자 3자 이상 (너무 짧은 패턴 제외)
        r'[0-9]{3,}[A-Za-z]{3,}',       # 숫자 3자 이상 + 영어 3자 이상
        r'[A-Za-z]{2,}[^A-Za-z0-9가-힣\s]{2,}',  # 영어 2자 이상 + 특수문자 2자 이상
        r'[^A-Za-z0-9가-힣\s]{2,}[A-Za-z]{2,}',  # 특수문자 2자 이상 + 영어 2자 이상
        r'[A-Za-z]{2,}-[0-9]+-[A-Za-z]{2,}',  # 영어 2자 이상 - 숫자 - 영어 2자 이상
        r'[^A-Za-z0-9가-힣\s]{3,}',    # 연속된 특수 문자 3자 이상
        # r'^(?=[^{}\(\*\[\]]*[}\(\*\[][^{}\(\*\[\]]*$)',  # 특정 기호가 하나만 포함된 문자열 필터링
        # r'[A-Za-z]{1,2}[^A-Za-z0-9가-힣\s]+[가-힣]',  # 영어 1~2자 + 특수문자 + 한글
        # r'[가-힣]+[^A-Za-z0-9가-힣\s]+[A-Za-z]{1,2}',  # 한글 + 특수문자 + 영어 1~2자
        # r'[가-힣]+[^A-Za-z0-9가-힣\s]+[가-힣]+[A-Za-z]',  # 한글 + 특수문자 + 한글 + 영어  
        # r'[^A-Za-z0-9가-힣\s]+[0-9]+[^A-Za-z0-9가-힣\s]+', # 특수문자 + 숫자 + 특수문자 
    ]
    
    has_abnormal_pattern = False
    for pattern in abnormal_patterns:
        if re.search(pattern, text_no_space):
            has_abnormal_pattern = True
            break
    
    # 필터링 조건 설정
    if korean_ratio < 0.6 or special_ratio > 0.1 or has_abnormal_pattern:
        noisy_texts.append(text)
        noisy_ids.append(text_id)
        if korean_ratio < 0.6:
            excluded_due_to_korean_ratio += 1
        if special_ratio > 0.1:
            excluded_due_to_special_ratio += 1
        if has_abnormal_pattern:
            excluded_due_to_abnormal_pattern += 1
        continue

    # 통과한 텍스트와 ID 저장
    normal_texts.append(text)
    normal_ids.append(text_id)

# 정상 텍스트와 노이즈 텍스트를 각각 데이터프레임으로 변환하여 저장
normal_texts_df = pd.DataFrame({'ID': normal_ids, 'text': normal_texts})
noisy_texts_df = pd.DataFrame({'ID': noisy_ids, 'text': noisy_texts})

normal_texts_df.to_csv('/data/ephemeral/data/normal_texts_ID.csv', index=False)
noisy_texts_df.to_csv('/data/ephemeral/data/noisy_texts_ID.csv', index=False)

print(f"총 {len(normal_texts)}개의 정상 텍스트를 추출하였습니다.")
print(f"총 {len(noisy_texts)}개의 노이즈 텍스트를 추출하였습니다.")
print(f"총 텍스트 수: {total_texts}")
print(f"한글 비율로 제외된 텍스트 수: {excluded_due_to_korean_ratio}")
print(f"특수 문자 비율로 제외된 텍스트 수: {excluded_due_to_special_ratio}")
print(f"이상 패턴으로 제외된 텍스트 수: {excluded_due_to_abnormal_pattern}")


총 1296개의 정상 텍스트를 추출하였습니다.
총 1504개의 노이즈 텍스트를 추출하였습니다.
총 텍스트 수: 2800
한글 비율로 제외된 텍스트 수: 748
특수 문자 비율로 제외된 텍스트 수: 1370
이상 패턴으로 제외된 텍스트 수: 297


# LLM 으로 학습하기 위해 노이즈 데이터 생성 구조와 비슷한 체계로 노이즈 생성

In [ ]:
import pandas as pd
import random
import string

# 데이터셋 로드
data = pd.read_csv('/data/ephemeral/jung/normal_texts.csv')  # 깨끗한 데이터셋 파일명

# 노이즈 데이터를 저장할 리스트
noisy_texts = []

# 특수 문자 포함한 아스키 문자 집합 생성
ascii_chars = string.printable 

def add_noise_to_text(text, min_ratio=0.1, max_ratio=0.5):
    text_chars = list(text)
    total_length = len(text_chars)
    if total_length == 0:
        return text  
    
    # 대체할 문자 수 결정 (10~50% 범위 내에서 랜덤하게 결정) => 80%는 노이즈가 너무 심해 LLM 이 제대로 학습하지 못하는 경향이 있으므로 제외
    replace_ratio = random.uniform(min_ratio, max_ratio)
    num_chars_to_replace = max(1, int(total_length * replace_ratio))  # 최소 1개는 교체

    # 대체할 위치를 랜덤으로 선택
    indices = list(range(total_length))
    random.shuffle(indices)
    indices_to_replace = indices[:num_chars_to_replace]
    
    # 문자 대체
    for idx in indices_to_replace:
        # 임의의 아스키 문자 선택
        random_ascii_char = random.choice(ascii_chars)
        text_chars[idx] = random_ascii_char
    
    noisy_text = ''.join(text_chars)
    return noisy_text

# 노이즈 데이터 생성
for text in data['text']:
    noisy_text = add_noise_to_text(text)
    noisy_texts.append(noisy_text)

# 노이즈-원본 데이터쌍 생성
data['noisy_text'] = noisy_texts

# 결과 저장
data[['noisy_text', 'text']].to_csv('noisy_clean_pairs.csv', index=False)
